In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).strftime("%Y%m%d")
today

'20250519'

## Import Connected TV TCs and add other device columns

### Group by main and sub categories to create `df_dev_component`

In [232]:
# Define available devices
available_devices = [
    'Android Mobile', 'Apple Mobile', 'Android TV', 'Apple TV', 
    'Fire TV', 'Smart TV', 'Vizio TV', 'Roku', 'Web'
]

# Import and process Connected TV test cases
df_connectedTV_raw = pd.read_csv("data/KOCOWA_4.0_tc_connectedTV.csv").rename(
    columns={"FireTV": "Fire TV", "ROKU": "Roku", "AndroidTV": "Android TV", "AppleTV": "Apple TV"}
).fillna(0).reset_index(drop=True)

df_connectedTV_raw["Web"] = False
df_connectedTV_raw["Apple Mobile"] = False
df_connectedTV_raw["Android Mobile"] = False
df_connectedTV_raw["Smart TV"] = True
df_connectedTV_raw["Vizio TV"] = True
df_connectedTV_raw["project_name"] = "KOCOWA 4.0"

# Import and process Web and Mobile test cases
df_webMobile_raw = pd.read_csv("data/KOCOWA_4.0_tc_webMobile_Edited.csv").rename(
    columns={"WEB": "Web", "Apple_Mobile": "Apple Mobile", "Android_Phone": "Android Mobile"}
).fillna(0).reset_index(drop=True)

df_webMobile_raw["Android TV"] = False
df_webMobile_raw["Apple TV"] = False
df_webMobile_raw["Fire TV"] = False
df_webMobile_raw["Smart TV"] = False
df_webMobile_raw["Vizio TV"] = False
df_webMobile_raw["Roku"] = False
df_webMobile_raw["project_name"] = "KOCOWA 4.0"

# Consolidate dataframes
df_K4_consolidated = pd.concat([df_connectedTV_raw, df_webMobile_raw], axis=0)

# Replace specific values in device columns
columns_to_check = df_K4_consolidated.columns[6:15]
df_K4_consolidated[columns_to_check] = df_K4_consolidated[columns_to_check].replace({
    "UNTESTED": True,
    "비대상": False
})

# Rename columns for consistency
df_K4_consolidated = df_K4_consolidated.rename(columns={
    "대분류": "main_category",
    "중분류": "sub_category",
    "소분류": "component",
    "Section": "scenario",
    "테스트 항목": "test_case",
    "기대결과": "expected_result"
})

# Strip whitespace from category columns
for col in ["main_category", "sub_category", "component"]:
    df_K4_consolidated[col] = df_K4_consolidated[col].astype(str).str.strip()

# Generate "scope_of_dev" column
def format_scope(main, sub, comp):
    if comp == "0":
        if sub == "0":
            return main
        else:
            return sub
    elif sub == "0":
        return main
    elif comp == sub:
        return sub
    else:
        return f"{sub} > {comp}"

df_K4_consolidated["scope_of_dev"] = df_K4_consolidated.apply(
    lambda row: format_scope(row["main_category"].strip(), row["sub_category"].strip(), row["component"].strip()), axis=1
)

# Group and aggregate data
device_columns = [
    'Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web', 
    'Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV'
]

df_dev_component = (
    df_K4_consolidated
    .groupby(['project_name', 'main_category', 'scope_of_dev'])
    .agg({
        'test_case': lambda x: '\n'.join(x),
        **{col: 'max' for col in device_columns}
    })
    .reset_index()
    .sort_values(by=['main_category', 'scope_of_dev'])
)

# Cleaning out the scopes with duplicated nested scopes
all_scopes = set(df_dev_component["scope_of_dev"])

for index, row in df_dev_component.iterrows():
    scope = row["scope_of_dev"]
    if any(other_scope.startswith(f"{scope} >") for other_scope in all_scopes):
        scope_rename = f"{scope} (General)"
        df_dev_component.at[index, "scope_of_dev"] = scope_rename

'''
nested_scopes = []
for scope in df_dev_component["scope_of_dev"]:
    # Check if any other scope starts with the current scope followed by ">"
    if any(other_scope.startswith(f"{scope} >") for other_scope in all_scopes):
        nested_scopes.append(scope)
'''

# Filter and reorder columns
col_orders = [
    'project_name', 'main_category', 'scope_of_dev', 'test_case', 
    'Fire TV', 'Roku', 'Android TV', 'Apple TV', 'Web', 
    'Apple Mobile', 'Android Mobile', 'Smart TV', 'Vizio TV'
]

# Test 1
df_dev_component_cleaned = df_dev_component[
    ~(df_dev_component["scope_of_dev"] == df_dev_component["main_category"])
][col_orders].copy()

/var/folders/lk/3z63ntmd7jx9rxfvb1tyzjb00000gn/T/ipykernel_24619/3844345128.py:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_K4_consolidated[columns_to_check] = df_K4_consolidated[columns_to_check].replace({


In [234]:
df_dev_component_cleaned[df_dev_component_cleaned["main_category"] == "Series Detailed Page"].sort_values(by=["main_category", "scope_of_dev"])

,project_name,main_category,scope_of_dev,test_case,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV
164,KOCOWA 4.0,Series Detailed Page,Continue POP UP,title text 확인\ntitle text 확인\nsub text 확인\nUse...,False,False,False,False,True,False,False,False,False
165,KOCOWA 4.0,Series Detailed Page,Description Area (General),Division Line 확인\nComma 확인,False,False,False,False,True,False,False,False,False
166,KOCOWA 4.0,Series Detailed Page,Description Area > Episode Counter,Episode Counter 확인\nEpisode Counter stringset ...,False,False,False,False,True,False,False,False,False
167,KOCOWA 4.0,Series Detailed Page,Description Area > More Button,More Button 확인\nMore Button stringset 확인\nMore...,False,False,False,False,True,False,False,False,False
168,KOCOWA 4.0,Series Detailed Page,Description Area > Rating icon,Rating icon 확인\nContent Rating icon 확인\nTV-Y R...,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,KOCOWA 4.0,Series Detailed Page,Webcomic Episode Rail > Oldest Button,Oldest Button 확인\nDefault Setting 확인\nOldest B...,False,False,False,False,True,False,False,False,False
279,KOCOWA 4.0,Series Detailed Page,Webcomic Episode Rail > Sorting Button,Sorting Button 확인\nDefault Setting 확인\nDownard...,False,False,False,False,True,False,False,False,False
280,KOCOWA 4.0,Series Detailed Page,Webcomic Episode Rail > thumnail image,thumnail image 확인\nLock icon 확인\nLock icon 노출 ...,False,False,False,False,True,False,False,False,False
281,KOCOWA 4.0,Series Detailed Page,Webcomics Screen (General),Webcomics Info box 확인\nWebcomics Info icon 확인\...,False,False,False,False,False,True,True,False,False


In [235]:
len(list(df_dev_component_cleaned.scope_of_dev.unique()))

318

In [236]:
df_K4_consolidated[df_K4_consolidated["main_category"] == "Series Detailed Page"].sort_values(by=["main_category", "sub_category"])

,main_category,sub_category,component,scenario,test_case,expected_result,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV,project_name,scope_of_dev
1,Series Detailed Page,0,0,Series Detailed Page,Scroll Up 동작 여부 확인,Scroll Up 동작하지 않는다,False,False,False,False,True,False,False,False,False,KOCOWA 4.0,Series Detailed Page
2,Series Detailed Page,0,0,Series Detailed Page,Scroll Down 동작 확인,Scroll Down 된다,False,False,False,False,True,False,False,False,False,KOCOWA 4.0,Series Detailed Page
3,Series Detailed Page,0,0,Series Detailed Page,Scroll Down 동작 여부 확인,Scroll Down 동작하지 않는다,False,False,False,False,True,False,False,False,False,KOCOWA 4.0,Series Detailed Page
4,Series Detailed Page,0,0,Series Detailed Page,Scroll Up 동작 확인,Scroll Up 된다,False,False,False,False,True,False,False,False,False,KOCOWA 4.0,Series Detailed Page
5,Series Detailed Page,0,0,Series Detailed Page,Background image 확인,Background image 노출된다,False,False,False,False,True,False,False,False,False,KOCOWA 4.0,Series Detailed Page
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,Series Detailed Page,Webcomics Screen,0,0,Free icon stringest 확인,"""Free"" text 출력된다",False,False,False,False,False,True,True,False,False,KOCOWA 4.0,Webcomics Screen
727,Series Detailed Page,Webcomics Screen,0,0,Free icon UI 확인,Free icon text 초록색으로 강조되어 노출된다,False,False,False,False,False,True,True,False,False,KOCOWA 4.0,Webcomics Screen
728,Series Detailed Page,Webcomics Screen,0,0,Last Viewed icon 확인,Last Viewed icon 노출된다,False,False,False,False,False,True,True,False,False,KOCOWA 4.0,Webcomics Screen
729,Series Detailed Page,Webcomics Screen,0,0,Last Viewed icon stringest 확인,Last Viewed text 출력된다,False,False,False,False,False,True,True,False,False,KOCOWA 4.0,Webcomics Screen


In [237]:
df_dev_component_cleaned

,project_name,main_category,scope_of_dev,test_case,Fire TV,Roku,Android TV,Apple TV,Web,Apple Mobile,Android Mobile,Smart TV,Vizio TV
0,KOCOWA 4.0,Contents,More Like This Screen,poster image 확인\nposter image focus 시 UI 확인\np...,True,True,True,True,False,False,False,True,True
1,KOCOWA 4.0,Contents,Series Detailed Page (General),리모컨 위 방향키 입력 시 동작 확인\n리모컨 아래 방향키 입력 시 동작 확인\n리...,True,True,True,True,False,False,False,True,True
2,KOCOWA 4.0,Contents,Series Detailed Page > Bookmarked Button,Bookmarked Button 확인\nBookmarked Button icon 확...,True,True,True,True,False,False,False,True,True
3,KOCOWA 4.0,Contents,Series Detailed Page > Continue Watching Button,Continue Watching Button 확인\nContinue Watching...,True,True,True,True,False,False,False,True,True
4,KOCOWA 4.0,Contents,Series Detailed Page > Details Section,Content Released Year 확인\nGenre text 확인\nDivis...,True,True,True,True,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,KOCOWA 4.0,VARIETY Channel Page,Sorting POP UP,Default Setting 확인\nCheck icon 확인\n리모컨 아래 방향키 ...,True,True,True,True,False,False,False,True,True
372,KOCOWA 4.0,VARIETY Channel Page,Top Area (General),Division Line 확인,True,True,True,True,False,False,False,True,True
373,KOCOWA 4.0,VARIETY Channel Page,Top Area > Sorting Button,Default Setting 확인\nSorting Button 확인\nSorting...,True,True,True,True,False,False,False,True,True
375,KOCOWA 4.0,WELCOME TO KOCOWA Page,Functional Area > Home Button,Home Button 확인\nHome Button stringset 확인\nHome...,True,True,True,True,False,False,False,True,True


### Initial export for checkup (csv and excel): processed_KOCOWA_4.0_tc_connectedTV

In [238]:
df_dev_component_cleaned.to_csv("data/processed/processed_KOCOWA_4.0_tc_connectedTV.csv", index=False)
# df_dev_component.to_excel("data/processed/processed_KOCOWA_4.0_tc_connectedTV.xlsx", index=False)